# Bases de datos con SQLAlchemy y SQLite

In [1]:
import sqlite3
from sqlalchemy import create_engine, Table, MetaData, select, or_, and_, func

In [2]:
# Creamos una base de datos y un cursor
conn_sqlite = sqlite3.connect('./biblioteca.db')
cursor = conn_sqlite.cursor()

In [3]:
# Creamos una tabla
cursor.execute("""CREATE TABLE escritor (
                    'id' INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
                    'nombre' TEXT NOT NULL,
                    'paterno' TEXT NOT NULL,
                    'materno' TEXT DEFAULT " ");""")
conn_sqlite.commit()

In [4]:
# Verificamos la creación de la tabla
engine = create_engine('sqlite:///biblioteca.db')
conn_sqlalchemy = engine.connect()
metadata = MetaData()
escritor = Table('escritor', metadata, autoload=True, autoload_with=engine)
print(escritor.columns.keys())

['id', 'nombre', 'paterno', 'materno']


In [5]:
# Introducimos algunos valores
cursor.execute("""INSERT INTO escritor (nombre, paterno) 
                   VALUES 
                   ('Hermann','Hesse'),
                   ('Jostein', 'Gaarder');""")
cursor.execute("""INSERT INTO escritor (nombre, paterno, materno)
                    VALUES
                    ('Jorge', 'Volpi', 'Escalante');""")
conn_sqlite.commit()

In [6]:
# Hacemos una consulta: aquellos escritores cuyo nombre sea Hermann o cuyo id sea 3
query = select([escritor])
query = query.where(
                or_(escritor.columns.nombre == 'Hermann', 
                    escritor.columns.id == 3))
results_proxy = conn_sqlalchemy.execute(query)
results = results_proxy.fetchall()
for result in results:
    print(result)

(1, 'Hermann', 'Hesse', ' ')
(3, 'Jorge', 'Volpi', 'Escalante')


In [7]:
# Creamos otra tabla
cursor.execute("""CREATE TABLE libro (
                    'id' INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
                    'nombre' TEXT NOT NULL,
                    'descripcion' TEXT NOT NULL DEFAULT " ",
                    'autor' INTEGER NOT NULL,
                    FOREIGN KEY(autor) REFERENCES escritor(id));""")
conn_sqlite.commit()

In [8]:
# Verificamos la creación de la tabla
libro = Table('libro', metadata, autoload=True, autoload_with=engine)
print(libro.columns.keys())

['id', 'nombre', 'descripcion', 'autor']


In [9]:
# Introducimos algunos valores
cursor.execute("""INSERT INTO libro (nombre, autor) 
                   VALUES 
                   ('Demian',1),
                   ('Siddhartha', 1),
                   ('El lobo estepario',1);""")
cursor.execute("""INSERT INTO libro (nombre, autor, descripcion)
                    VALUES
                    ('El Mundo de Sofia', 2, 'Novela sobre la historia de la filosofía');""")
conn_sqlite.commit()

In [10]:
# Hacemos otra consulta: los libros cuyo titulo empieze con E
query = select([libro])
query = query.where(libro.columns.nombre.like('E%'))
results_proxy = conn_sqlalchemy.execute(query)
results = results_proxy.fetchall()
for result in results:
    print(result)

(3, 'El lobo estepario', ' ', 1)
(4, 'El Mundo de Sofia', 'Novela sobre la historia de la filosofía', 2)


In [11]:
# Hacemos un inner join entre las tablas
query = select([escritor, libro])
query = query.select_from(escritor.join(libro, escritor.columns.id == libro.columns.autor))
results_proxy = conn_sqlalchemy.execute(query)
results = results_proxy.fetchall()
for result in results:
    print(result)

(1, 'Hermann', 'Hesse', ' ', 1, 'Demian', ' ', 1)
(1, 'Hermann', 'Hesse', ' ', 2, 'Siddhartha', ' ', 1)
(1, 'Hermann', 'Hesse', ' ', 3, 'El lobo estepario', ' ', 1)
(2, 'Jostein', 'Gaarder', ' ', 4, 'El Mundo de Sofia', 'Novela sobre la historia de la filosofía', 2)


In [12]:
# ¿Cuantos libros tiene Hermann Hesse registrados?
query = select([func.sum(escritor.columns.id)])
query = query.select_from(escritor.join(libro, escritor.columns.id == libro.columns.autor))
query = query.where(
                and_(escritor.columns.nombre == 'Hermann', 
                     escritor.columns.paterno == 'Hesse'))
results_proxy = conn_sqlalchemy.execute(query)
results = results_proxy.scalar()
results

3